In [2]:
# %load ./object-detection-deep-learning/deep_learning_object_detection.py
# USAGE
# python deep_learning_object_detection.py --image images/example_01.jpg \
#	--prototxt MobileNetSSD_deploy.prototxt.txt --model MobileNetSSD_deploy.caffemodel

# import the necessary packages
import numpy as np
import argparse
import cv2
import matplotlib.pyplot as plt
import os
import ntpath

In [66]:
# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe("./object-detection-deep-learning/MobileNetSSD_deploy.prototxt.txt",
                               "./object-detection-deep-learning/MobileNetSSD_deploy.caffemodel")

def format_image(path_img):
# load the input image and construct an input blob for the image
# by resizing to a fixed 300x300 pixels and then normalizing it
# (note: normalization is done via the authors of the MobileNet SSD
# implementation)
    image = cv2.imread(path_img)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)
    return blob, image, h, w 

def perform_detection(blob) :
    
    net.setInput(blob)
    detections = net.forward()
    return detections

def filter_detections(detections, confidence_th, image, h,w, basename, saving_path):
    # loop over the 
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > confidence_th:
            # extract the index of the class label from the `detections`,
            # then compute the (x, y)-coordinates of the bounding box for
            # the object
            idx = int(detections[0, 0, i, 1])
            if idx == 3 :
                
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                # display the prediction
                label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
                print("[INFO] {}".format(label))
                cv2.rectangle(image, (startX, startY), (endX, endY),
                    COLORS[idx], 2)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(image, label, (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
                
                path_save = os.path.join(saving_path, basename)
                if os.path.exists(path_save+".png"):
                    cv2.imwrite(path_save+"_"+str(i)+".png",image[startY:endY,startX:endX])
                else :
                    cv2.imwrite(path_save+".png",image[startY:endY,startX:endX])
# show the output image
#     return plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    


[INFO] loading model...


In [90]:
def return_cropped_image(path_img, saving_path):
    confidence_th = 0.8
    basename = ntpath.basename(path_img)
    blob, image, h, w = format_image(path_img)
    detections = perform_detection(blob)
    filter_detections(detections, confidence_th, image, h, w, basename, saving_path)


source_images = "./bird_dataset"
cropped_images = "./bird_dataset_cropped"



def loop_over_images(srs=source_images, 
                     target= cropped_images) :
    for type_ in ["train_images","val_images"]:
        path_srs_0 = os.path.join(srs, type_)
        path_target_0 = os.path.join(target, type_)
        classes_names = [x[1] for x in os.walk(path_srs_0)][0]
        for class_ in classes_names :
            path_srs_1 = os.path.join(path_srs_0, class_)
            path_target_1 = os.path.join(path_target_0, class_)
            if not os.path.exists(path_target_1):
                os.makedirs(path_target_1)
            images_paths = [x[2] for x in os.walk(path_srs_1)][0]
            for img_path in images_paths :
                return_cropped_image(os.path.join(path_srs_1,img_path), path_target_1)

In [91]:
loop_over_images()

[INFO] bird: 99.89%
[INFO] bird: 99.99%
[INFO] bird: 98.98%
[INFO] bird: 84.13%
[INFO] bird: 83.55%
[INFO] bird: 98.55%
[INFO] bird: 84.51%
[INFO] bird: 98.84%
[INFO] bird: 86.36%
[INFO] bird: 99.57%
[INFO] bird: 86.80%
[INFO] bird: 100.00%
[INFO] bird: 94.88%
[INFO] bird: 84.71%
[INFO] bird: 99.71%
[INFO] bird: 99.70%
[INFO] bird: 99.95%
[INFO] bird: 99.99%
[INFO] bird: 99.82%
[INFO] bird: 98.87%
[INFO] bird: 99.97%
[INFO] bird: 99.30%
[INFO] bird: 97.02%
[INFO] bird: 100.00%
[INFO] bird: 99.98%
[INFO] bird: 99.99%
[INFO] bird: 99.99%
[INFO] bird: 99.63%
[INFO] bird: 92.67%
[INFO] bird: 99.90%
[INFO] bird: 99.90%
[INFO] bird: 85.34%
[INFO] bird: 99.00%
[INFO] bird: 94.20%
[INFO] bird: 99.97%
[INFO] bird: 95.02%
[INFO] bird: 99.58%
[INFO] bird: 99.38%
[INFO] bird: 99.40%
[INFO] bird: 97.10%
[INFO] bird: 99.40%
[INFO] bird: 99.96%
[INFO] bird: 99.34%
[INFO] bird: 99.99%
[INFO] bird: 93.90%
[INFO] bird: 99.92%
[INFO] bird: 99.96%
[INFO] bird: 99.94%
[INFO] bird: 99.85%
[INFO] bird: 99.98

[INFO] bird: 100.00%
[INFO] bird: 99.68%
[INFO] bird: 98.93%
[INFO] bird: 100.00%
[INFO] bird: 100.00%
[INFO] bird: 91.32%
[INFO] bird: 99.99%
[INFO] bird: 100.00%
[INFO] bird: 99.99%
[INFO] bird: 99.84%
[INFO] bird: 100.00%
[INFO] bird: 99.22%
[INFO] bird: 100.00%
[INFO] bird: 99.98%
[INFO] bird: 99.99%
[INFO] bird: 99.87%
[INFO] bird: 100.00%
[INFO] bird: 99.89%
[INFO] bird: 100.00%
[INFO] bird: 99.95%
[INFO] bird: 96.29%
[INFO] bird: 99.29%
[INFO] bird: 99.98%
[INFO] bird: 99.98%
[INFO] bird: 99.91%
[INFO] bird: 99.79%
[INFO] bird: 99.77%
[INFO] bird: 99.76%
[INFO] bird: 99.84%
[INFO] bird: 99.97%
[INFO] bird: 100.00%
[INFO] bird: 100.00%
[INFO] bird: 100.00%
[INFO] bird: 84.94%
[INFO] bird: 99.99%
[INFO] bird: 99.99%
[INFO] bird: 99.98%
[INFO] bird: 99.33%
[INFO] bird: 99.98%
[INFO] bird: 99.98%
[INFO] bird: 99.99%
[INFO] bird: 99.54%
[INFO] bird: 99.99%
[INFO] bird: 100.00%
[INFO] bird: 99.95%
[INFO] bird: 98.64%
[INFO] bird: 93.64%
[INFO] bird: 100.00%
[INFO] bird: 100.00%
[INFO]

[INFO] bird: 99.98%
[INFO] bird: 99.03%
[INFO] bird: 99.90%
[INFO] bird: 99.13%
[INFO] bird: 97.86%
[INFO] bird: 99.96%
[INFO] bird: 99.98%
[INFO] bird: 94.39%
[INFO] bird: 97.94%
[INFO] bird: 98.44%
[INFO] bird: 82.07%
[INFO] bird: 99.92%
[INFO] bird: 99.98%
[INFO] bird: 99.11%
[INFO] bird: 88.41%
[INFO] bird: 100.00%
[INFO] bird: 96.50%
[INFO] bird: 97.18%
[INFO] bird: 82.86%
[INFO] bird: 99.97%
[INFO] bird: 92.66%
[INFO] bird: 99.78%
[INFO] bird: 91.15%
[INFO] bird: 97.53%
[INFO] bird: 89.76%
[INFO] bird: 99.25%
[INFO] bird: 80.11%
[INFO] bird: 99.97%
[INFO] bird: 94.49%
[INFO] bird: 99.99%
[INFO] bird: 100.00%
[INFO] bird: 98.44%
[INFO] bird: 96.08%
[INFO] bird: 99.74%
[INFO] bird: 97.15%
[INFO] bird: 95.40%
[INFO] bird: 81.98%
[INFO] bird: 99.35%
[INFO] bird: 99.97%
[INFO] bird: 99.97%
[INFO] bird: 99.99%
[INFO] bird: 99.99%
[INFO] bird: 99.92%
[INFO] bird: 99.89%
[INFO] bird: 100.00%
[INFO] bird: 99.95%
[INFO] bird: 99.98%
[INFO] bird: 100.00%
[INFO] bird: 99.95%
[INFO] bird: 99.